In [2]:
import pandas as pd
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
from datetime import datetime
import itertools

# 한글 깨짐
font_path = 'C:\WINDOWS\FONTS\MALGUN.ttf'
font = font_manager.FontProperties(fname = font_path).get_name()
rc('font', family = font)

In [5]:
# 출입일시 csv 파일 불러오기
lib_entry_exit = pd.read_csv("./데이터_도서관출입.csv", encoding = "cp949", low_memory=False)

# 출입일시와 출/입 정보를 인덱싱한 새로운 데이터 프레임 생성
entry_or_exit_and_time = lib_entry_exit.loc[:,['출입일시','출입구']]
# non값이 있는 행을 모두 삭제(전처리)
entry_or_exit_and_time = entry_or_exit_and_time.dropna(how='all')
# 출입일시 정렬
entry_or_exit_and_time = entry_or_exit_and_time.sort_values(by='출입일시')

# 출입 날짜만 가진 새로운 행을 추가하기 위해
# 일단, 출입일시를 새로만든 날짜행에 모두 추가
entry_or_exit_and_time['날짜'] = entry_or_exit_and_time["출입일시"]

# 날짜만을 추출하는 함수 정의
def only_date(values):
    return values[0:10]

# 날짜 행에 날짜만을 저장
entry_or_exit_and_time['날짜'] = entry_or_exit_and_time['날짜'].apply(only_date)

# 같은 날짜 별로 groupby 진행
a_day_list = entry_or_exit_and_time.groupby("날짜")

# 날짜별로 리스트 생성
list_day_total = []
for key, group in a_day_list:
    list_for_a_day_info = group.values.tolist()
    list_day_total.append(list_for_a_day_info)

In [6]:
# most_crowed_time_in_lib 함수
# 일별로 모은 리스트 데이터를 대상(->매개변수)으로 가장 혼잡한 시간을 리턴해주는 함수 생성

# 도서관 시간 6시 ~ 23시
# lib_AtoB : A시 00분부터 A시 59분까지의 기록
def most_crowed_time_in_lib(list_day):
    lib_6to7 = []
    lib_7to8 = []
    lib_8to9 = []
    lib_9to10 = []
    lib_10to11 = []

    lib_11to12 = []
    lib_12to13 = []
    lib_13to14 = []
    lib_14to15 = []
    lib_15to16 = []

    lib_16to17 = []
    lib_17to18 = []
    lib_18to19 = []
    lib_19to20 = []
    lib_20to21 = []

    lib_21to22 = []
    lib_22to23 = []

    lib_AtoB = [
        lib_6to7,
        lib_7to8,
        lib_8to9,
        lib_9to10,
        lib_10to11,
        lib_11to12,
        lib_12to13,
        lib_13to14,
        lib_14to15,
        lib_15to16,
        lib_16to17,
        lib_17to18,
        lib_18to19,
        lib_19to20,
        lib_20to21,
        lib_21to22,
        lib_22to23
    ]

    # 시간별로 도서관 출입 (입구 - 출구)을 저장할 리스트
    lib_time_cnt = []

    for i in list_day:
        datetime_d = datetime.strptime(i[0], '%Y/%m/%d %H:%M:%S')
        if datetime_d.hour == 6:
            lib_6to7.append([i[0],i[1]])
        elif datetime_d.hour == 7:
            lib_7to8.append([i[0],i[1]])
        elif datetime_d.hour == 8:
            lib_8to9.append([i[0],i[1]])
        elif datetime_d.hour == 9:
            lib_9to10.append([i[0],i[1]])
        elif datetime_d.hour == 10:
            lib_10to11.append([i[0],i[1]])
        elif datetime_d.hour == 11:
            lib_11to12.append([i[0],i[1]])
        elif datetime_d.hour == 12:
            lib_12to13.append([i[0],i[1]])
        elif datetime_d.hour == 13:
            lib_13to14.append([i[0],i[1]])
        elif datetime_d.hour == 14:
            lib_14to15.append([i[0],i[1]])
        elif datetime_d.hour == 15:
            lib_15to16.append([i[0],i[1]])
        elif datetime_d.hour == 16:
            lib_16to17.append([i[0],i[1]])
        elif datetime_d.hour == 17:
            lib_17to18.append([i[0],i[1]])
        elif datetime_d.hour == 18:
            lib_18to19.append([i[0],i[1]])
        elif datetime_d.hour == 19:
            lib_19to20.append([i[0],i[1]])
        elif datetime_d.hour == 20:
            lib_20to21.append([i[0],i[1]])
        elif datetime_d.hour == 21:
            lib_21to22.append([i[0],i[1]])
        elif datetime_d.hour == 22:
            lib_22to23.append([i[0],i[1]])


    for i in lib_AtoB:
        lib_cnt = 0
        for j in i:
            if (j[1] == '입구'):
                lib_cnt += 1
            elif (j[1] == '출구'):
                lib_cnt -= 1
        lib_time_cnt.append(lib_cnt)

    # 시 단위로 축적시키기(accumulate 사용)
    lib_accumulate = itertools.accumulate(lib_time_cnt)

    # 누적된 사람 수 리스트에 저장시키기
    list_accumulate = []
    for item in lib_accumulate:
        list_accumulate.append(item)
        # 6시 부터 22시까지 출력 됨(인덱스 = 시간 - 6)

    # 최대값(사람 수) 찾기
    max_lib = max(list_accumulate)
    max_time = list_accumulate.index(max_lib) + 6
    # 시간 = 인덱스 + 6
    
    return max_time

In [7]:
# most_crowed_time_list는 과거(이전날)의 data를 통해 가장 혼잡한 시간을 예측한 리스트
# 가정 ) 이 전날에 가장 혼잡헀던 시간이 그 다음날에도 혼잡할 것이라고 예측
# 이전날의 데이터를 가지고 다음날의 혼잡한 시간을 예측하기에
# 가장 첫번째 most_crowed_time_list의 값은 pop

most_crowed_time_list = []

for i in range(len(list_day_total)-1):
    most_crowed_time_list.append(most_crowed_time_in_lib(list_day_total[i]))

most_crowed_time_list.pop(0)

print(most_crowed_time_list)

# print(most_crowed_time_list.count(13)) 128
# print(most_crowed_time_list.count(14)) 253
# print(most_crowed_time_list.count(15)) 946 => 가장 많음
# print(most_crowed_time_list.count(16)) 373

# 15~16시의 도서관에 가장 혼잡할 확률이 높음

[15, 15, 13, 15, 15, 15, 15, 15, 15, 15, 16, 15, 16, 15, 15, 13, 16, 15, 16, 16, 15, 15, 15, 16, 16, 16, 16, 15, 15, 15, 16, 16, 16, 16, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 15, 16, 19, 16, 19, 16, 16, 15, 16, 15, 16, 15, 15, 15, 13, 16, 15, 13, 15, 15, 16, 13, 16, 15, 16, 15, 15, 15, 13, 15, 12, 16, 12, 15, 15, 16, 16, 13, 16, 12, 15, 13, 16, 16, 13, 16, 16, 15, 15, 16, 16, 16, 16, 16, 16, 16, 15, 16, 19, 16, 15, 15, 15, 15, 16, 15, 15, 13, 15, 15, 14, 15, 15, 15, 15, 15, 13, 16, 16, 14, 15, 15, 15, 14, 15, 16, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 15, 16, 14, 15, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 15, 15, 15, 15, 15, 15, 15, 16, 15, 15, 15, 15, 15, 15, 14, 15, 14, 15, 15, 16, 14, 15, 14, 14, 15, 13, 15, 15, 13, 15, 15, 13, 15, 16, 12, 16, 15, 15, 13, 16, 16, 12, 16, 15, 15, 15, 16, 16, 12, 16, 15, 15, 15, 15, 16, 15, 15, 16, 15, 15, 16, 16, 15, 16, 16, 16, 15, 15, 16, 18, 19, 16, 16, 16, 15, 16, 16, 16, 19, 16, 15, 15, 16, 19, 16, 12, 16, 15, 16, 15, 15,